In [1]:
import sys
sys.path.append('/Users/HeeyunLee/Developer/cccc/functions/source')

print(sys.path)

['/Users/HeeyunLee/Developer/cccc/functions/source/tests', '/Users/HeeyunLee/.vscode/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles/vscode_datascience_helpers', '/Users/HeeyunLee/.vscode/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles', '/Users/HeeyunLee/.vscode/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles/lib/python', '/Users/HeeyunLee/opt/anaconda3/lib/python38.zip', '/Users/HeeyunLee/opt/anaconda3/lib/python3.8', '/Users/HeeyunLee/opt/anaconda3/lib/python3.8/lib-dynload', '', '/Users/HeeyunLee/opt/anaconda3/lib/python3.8/site-packages', '/Users/HeeyunLee/opt/anaconda3/lib/python3.8/site-packages/aeosa', '/Users/HeeyunLee/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/HeeyunLee/.ipython', '/Users/HeeyunLee/Developer/cccc/functions/source']


In [2]:
from test_configuration import plaid_client, firestore_client
from typing import Dict, Any, Union, List
import json
from datetime import datetime

from google.cloud.firestore import CollectionReference

from plaid.model.products import Products
from plaid import ApiException
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
from plaid.model.account_balance import AccountBalance
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
from plaid.model.item_public_token_exchange_response import ItemPublicTokenExchangeResponse
from plaid.model.sandbox_public_token_create_response import SandboxPublicTokenCreateResponse

### Account Balance Get
from plaid.model.accounts_balance_get_request import AccountsBalanceGetRequest
from plaid.model.accounts_get_response import AccountsGetResponse

In [3]:
def sandbox_public_token_create(institution_id: str) -> Dict[str, Any]:
    try:
        request = SandboxPublicTokenCreateRequest(
            institution_id=institution_id,
            initial_products=[Products('transactions')],
        )

        response: SandboxPublicTokenCreateResponse = plaid_client.sandbox_public_token_create(
            request
        )
        response_dict = response.to_dict()

        return response_dict
    except ApiException as e:
        exceptions: dict = json.loads(e.body)

        return exceptions

In [ ]:
sandbox_public_token_create('ins_3')

In [4]:
def public_token_exchange(data: Dict[str, Any]) -> Dict[str, Any]:
    try:
        # Get public_token FOR SANDBOX
        institution_id: str = data.get('institution_id')
        public_token_response = sandbox_public_token_create(institution_id)
        public_token: Union[str, None] = public_token_response.get('public_token')
        print(f'public_token is {public_token}')

        # # Get public_token FOR DEVELOPMENT
        # public_token: Union[str, None] = data.get('public_token')


        if public_token is None:
            return {'error_code': 404, 'error_message': 'public_token is None'}

        # Make Request to exchange public_token_to_access_token
        request = ItemPublicTokenExchangeRequest(public_token)
        response: ItemPublicTokenExchangeResponse = plaid_client.item_public_token_exchange(request)
        response_dict = response.to_dict()
        print(f'exchange response dict: \n{response_dict}')

        return response_dict
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
exchange_response = public_token_exchange({'institution_id': 'ins_3'})

In [5]:
''' Retrieve real-time balance data

    The `/accounts/balance/get` endpoint returns the real-time balance for each of 
    an Item's accounts. While other endpoints may return a balance object, only 
    `/accounts/balance/get` forces the available and current balance fields to be 
    refreshed rather than cached. This endpoint can be used for existing Items 
    that were added via any of Plaid’s other products. This endpoint can be used 
    as long as Link has been initialized with any other product, balance itself 
    is not a product that can be used to initialize Link.
'''
def accounts_balance_get(access_token: str):
    try:
        request = AccountsBalanceGetRequest(access_token)
        response: AccountsGetResponse = plaid_client.accounts_balance_get(request)

        return response.to_dict()
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions


In [6]:
def update_accounts(uid: str, ins_id: str, accounts: List[dict]) -> Dict[str, Any]:
    try:
        user_doc = firestore_client.collection('users').document(uid)
        accounts_ref: CollectionReference = user_doc.collection('accounts')

        # Update Accounts in `Accounts` collections
        for account in accounts:
            print(f'Account: {account}')

            # Update Accounts Collections
            account_id: str = account.get('account_id')
            print(f'accountId: {account_id}')

            account_doc_ref = accounts_ref.document(account_id)
            account_snapshot = account_doc_ref.get()

            balances: Union[AccountBalance, None] = account.get('balances')
            print(f'Balances {balances}')

            data = {
                'account_id': account.get('account_id'),
                'balances': {
                    'available': balances.get('available'),
                    'current': balances.get('current'),
                    'iso_currency_code': balances.get('iso_currency_code'),
                    'limit': balances.get('limit'),
                    'unofficial_currency_code': balances.get('unofficial_currency_code'),
                },
                'mask': account.get('mask'),
                'name': account.get('name'),
                'official_name': account.get('official_name'),
                'subtype': str(account.get('subtype')),
                'type': str(account.get('type')),
                'verification_status': account.get('verification_status'),
                'account_last_synced_time': datetime.now(),
                'institution_id': ins_id,
            }

            if account_snapshot.exists:
                account_doc_ref.update(data)
            else:
                account_doc_ref.set(data)

            result = {
                'status': 200,
                'message': 'successfully updated accounts data',
            }

        return result
    except Exception as e:
        result = {
            'status': 404,
            'error_message': str(e),
        }

        return result

In [7]:
def update_user_secrets(uid: str, access_token: str) -> Dict[str, Any]:
    try:
        user_doc = firestore_client.collection('users').document(uid)
        secrets_collection: CollectionReference = user_doc.collection('secrets')
        user_secret_doc_ref = secrets_collection.document(uid)
        user_secret_doc_snapshot = user_secret_doc_ref.get()            
        
        if user_secret_doc_snapshot.exists:
            secret_doc = user_secret_doc_snapshot.to_dict()
            access_tokens: List[str] = secret_doc.get('plaid_access_tokens')
            print(f'old access tokens: {access_tokens}')

            access_tokens.append(access_token)
            print(f'new access tokens {access_tokens}')

            user_secret_doc_ref.update({'plaid_access_tokens': access_tokens})
        else:
            user_secret_doc_ref.create({'plaid_access_tokens': [access_token]})

        result = {
            'status': 200,
            'message': 'Successfully updated access_tokens',
        }

        return result
    except Exception as e:
        result = {
            'status': 404,
            'error': str(e),
        }
        
        return result

In [8]:
''' Link and Connect

    After a user connect a bank account and gets the `public_token`,
    this function is called to 
    1. Exchange `public_token` to `access_token`
    2. Using the `access_token`, call /accounts/balance/get
    3. Using the accounts info, update Firebase Firestore database
'''

def link_and_connect(request: Dict[str, Any]):

    # # TODO: uncomment below for release
    # # Parsing data from request to get `uid` and `public_token`
    # data = request.data
    # data_dict: dict = json.loads(data)
    # public_token: Union[str, None] = data.get('public_token')
    # uid: Union[str, None] = data_dict.get('uid')   

    uid: Union[str, None] = request.get('uid') 

    # 1. Get access_token by using `item_public_token_exchange`
    token_exchange_response = public_token_exchange(request)
    access_token = token_exchange_response.get('access_token')
    print(f'Got access_token: {access_token}')

    # If access_token is None, return error
    if access_token is None:
        error_code = token_exchange_response.get('error_code')
        error_message = token_exchange_response.get('error_message')
        print(f'Error code: {error_code}')
        print(f'Error message: {error_message}')

        # # TODO: uncomment below for release
        # return make_response(jsonify(error_code=error_code, error_message=error_message), 404)

    # 2. Else, call /accounts/balance/get
    balances_get_response = accounts_balance_get(access_token)
    accounts: Union[List[Dict], None] = balances_get_response.get('accounts')
    print(f'balances got: {balances_get_response}')

    # If accounts is None, return error
    if accounts is None:
        error_code = balances_get_response.get('error_code')
        error_message = balances_get_response.get('error_message')
        print(f'Error code: {error_code}')
        print(f'Error message: {error_message}')

        # # TODO: uncomment below for release
        # return make_response(jsonify(error_code=error_code, error_message=error_message), 404)

    # 3. Update [Account] collection if hot accounts
    print(f'Got {len(accounts)} of accounts')

    item: Union[Dict, None] = balances_get_response.get('item')
    institution_id: Union[str, None] = item.get('institution_id')
    update_account_result = update_accounts(uid, institution_id, accounts)
    print(f'update_account_result: {update_account_result}')

    # 4. Update user's secret keys with new access_token
    update_plaid_tokens_result = update_user_secrets(uid, access_token)
    print(f'update_plaid_tokens_result: {update_plaid_tokens_result}')

    # Get the status of these two firestore updates, and return
    update_account_status = update_account_result.get('status')
    update_access_token_status = update_plaid_tokens_result.get('status')
    print(f'''
        update_account_status: {update_account_status}
        update_access_token_status: {update_access_token_status}
    ''')

    # # TODO: uncomment below for release
    # result = jsonify(
    #     update_account=update_account_status,
    #     update_access_token=update_access_token_status,
    # )

    # return make_response(result)

In [10]:
data = {
    'institution_id': 'ins_3',
    'uid': 'VTVaLAqXC4VP6FS8lpRP1eCEV0G3',
}

# link_and_connect(data)